In [ ]:
!pip install langchain_google_vertexai

In [ ]:
!pip install langchain_community

In [ ]:
!pip install neo4j

In [ ]:
!pip install tiktoken

In [ ]:
!nvidia-smi

In [ ]:
!pip install ipython-cypher


In [ ]:
%load_ext cypher

## Neo4j Advanced rag 
https://github.com/langchain-ai/langchain/tree/master/templates/neo4j-advanced-rag this is has open ai reference and below you can find the gcp model implementation

Future work

Comparitive study with https://python.langchain.com/v0.1/docs/use_cases/graph/constructing/ approach

![Architecture](./Architecture.png)

In [3]:
from pathlib import Path
from typing import List, Optional
from langchain_google_vertexai import (
    VertexAI,
    ChatVertexAI,
    VertexAIEmbeddings,
    VectorSearchVectorStore,
)
from google.cloud import aiplatform
from langchain_community.vectorstores import Neo4jVector

from langchain_community.document_loaders import TextLoader
from langchain_community.graphs import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_text_splitters import TokenTextSplitter
from neo4j.exceptions import ClientError
import os
from vertexai.generative_models import GenerativeModel, Image
from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput
from langchain_google_vertexai import ChatVertexAI, HarmBlockThreshold, HarmCategory
from unstructured.partition.pdf import partition_pdf
from langchain_text_splitters import CharacterTextSplitter
from typing import List, Tuple
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField, RunnableParallel,RunnableLambda
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
import base64
from langchain_core.messages import AIMessage, HumanMessage
import typing


In [4]:
url = "bolt://url"
username ="neo4j"
password = "password"
os.environ["NEO4J_URI"] = url
os.environ["NEO4J_USERNAME"] = username
os.environ["NEO4J_PASSWORD"] = password

In [5]:
PROJECT_ID = ""  # @param {type:"string"}
LOCATION = ""  # @param {type:"string"}
aiplatform.init(project=PROJECT_ID, location=LOCATION)


In [6]:
# MATCH (n) MATCH ()-[r]->() RETURN n, r

In [7]:
# MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n, r;

In [8]:
pdf_folder_path = "/home/path/rag/data/image"
pdf_file_name = "/home/path/rag/data/filename.pdf"
Pdf_title="filename"

In [9]:
image_pdf_folder_path = "/home/path/rag/data/images"
pdf_file_name = "/home/path/rag/data/filename.pdf"

# Extract images, tables, and chunk text from a PDF file.
raw_pdf_elements = partition_pdf(
    filename=pdf_file_name,
    extract_images_in_pdf=False,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=pdf_folder_path,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_output_dir=image_pdf_folder_path,
    extract_image_block_to_payload=False
)

2024-08-03 16:42:48.530937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 16:42:48.548337: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 16:42:48.553289: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-03 16:42:48.567324: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at microsoft/tab

In [10]:


# Categorize extracted elements from a PDF into tables and texts.
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))

In [11]:
# Optional: Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=5000, chunk_overlap=0
)

In [12]:
MODEL_NAME = "gemini-1.0-pro-vision"


# Generate summaries of text elements
def generate_text_summaries(
    texts: List[str], tables: List[str], summarize_texts: bool = False
) -> Tuple[List, List]:
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a elaborate concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = PromptTemplate.from_template(prompt_text)
    empty_response = RunnableLambda(
        lambda x: AIMessage(content="Error processing document")
    )
    # Text summary chain
    model = VertexAI(
        temperature=0, model_name=MODEL_NAME, max_output_tokens=1024
    ).with_fallbacks([empty_response])
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts:
        if summarize_texts:
            text_summaries = summarize_chain.batch(texts, {"max_concurrency": 1})
        else:
            text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 1})

    return text_summaries, table_summaries


# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts, tables, summarize_texts=True
)

In [13]:
def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Make image summary"""
    model = ChatVertexAI(model_name="gemini-pro-vision", max_output_tokens=1024)

    msg = model(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval.
    If it's a table, extract all elements of the table.
    If it's a graph, explain the findings in the graph.
    Do not include any numbers that are not mentioned in the image.
    """

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries


# Image summaries
img_base64_list, image_summaries = generate_img_summaries(image_pdf_folder_path)

/opt/conda/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [14]:
combine_all_texts = text_summaries + table_summaries + image_summaries 
joined_texts = " ".join(combine_all_texts)
texts_4k_token = text_splitter.split_text(joined_texts)

In [15]:
len(texts_4k_token)

4

In [ ]:
docs = text_splitter.create_documents(texts_4k_token)
print(docs[0].page_content)

In [17]:
len(docs)

4

In [18]:
# Embeddings & LLM models
# embeddings = OpenAIEmbeddings()
embedding_dimension = 768
# llm = ChatOpenAI(temperature=0)

# Loading Gemini Pro Vision Model
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")
# Initializing embedding model
EMBEDDING_MODEL =VertexAIEmbeddings(model_name="textembedding-gecko@003", dimension=768) #TextEmbeddingModel.from_pretrained("textembedding-gecko@003")

# Loading Gemini Pro Model
llm = ChatVertexAI(
    model_name="gemini-1.0-pro-vision",
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },
)


In [291]:
parent_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
child_splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=24)
parent_documents = parent_splitter.split_documents(docs)

In [292]:
len(parent_documents)

11

In [293]:
embedding_dimension = 768

In [294]:
graph = Neo4jGraph()

In [295]:
for i, parent in enumerate(parent_documents):
    child_documents = child_splitter.split_documents([parent])
    print(len(parent.page_content))

9473
9559
5097
8972
8430
4056
9175
9317
3676
9110
5388


In [ ]:
for i, parent in enumerate(parent_documents):
    child_documents = child_splitter.split_documents([parent])
    print(len(parent.page_content))
    params = {
        "parent_text": parent.page_content,
        "parent_id": i,
        "parent_embedding": EMBEDDING_MODEL.embed_query(parent.page_content), #EMBEDDING_MODEL.get_embeddings([parent.page_content])[0].values,
        "children": [
            {
                "text": c.page_content,
                "id": f"{i}-{ic}",
                "embedding": EMBEDDING_MODEL.embed_query(c.page_content) #EMBEDDING_MODEL.get_embeddings([c.page_content])[0].values,
            }
            for ic, c in enumerate(child_documents)
        ],
    }
     # Ingest data
    graph.query(
        """
    MERGE (p:Parent {id: $parent_id})
    SET p.text = $parent_text
    WITH p
    CALL db.create.setVectorProperty(p, 'embedding', $parent_embedding)
    YIELD node
    WITH p 
    UNWIND $children AS child
    MERGE (c:Child {id: child.id})
    SET c.text = child.text
    MERGE (c)<-[:HAS_CHILD]-(p)
    WITH c, child
    CALL db.create.setVectorProperty(c, 'embedding', child.embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )
    # Create vector index for child
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('parent_document', "
            "'Child', 'embedding', 768, 'cosine')",
            {},
        )
    except ClientError:  # already exists
        pass
    # Create vector index for parents
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('typical_rag', "
            "'Parent', 'embedding', 768, 'cosine')",
            {},
        )
    except ClientError:  # already exists
        pass
# CALL db.index.vector.createNodeIndex('parent_document', 'Child', 'embedding', 768, 'cosine')

![neo4j](./neo4j3.png)

![neo4j](./neo4j2.png)

In [297]:
llm

ChatVertexAI(client=<google.cloud.aiplatform_v1beta1.services.prediction_service.client.PredictionServiceClient object at 0x7f4b0c18ed50>, project='edl-idaas-rnd-platform-d5ae', model_name='gemini-1.0-pro-vision', model_family=<GoogleModelFamily.GEMINI: '1'>, full_model_name='projects/edl-idaas-rnd-platform-d5ae/locations/us-central1/publishers/google/models/gemini-1.0-pro-vision', client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None, 'universe_domain': None}, default_metadata=(), temperature=0.0)

In [298]:
# Ingest frequently asked questions  questions


class Questions(BaseModel):
    """Generating hypothetical questions about text."""

    questions: List[str] = Field(
        ...,
        description=(
            "Generated hypothetical questions based on  the information from the text"
        ),
    )


In [299]:

questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            (
                "Use the given format to generate frequently asked questions from the "
                "following input: {input}"
            ),
        ),
    ]
)
question_chain = questions_prompt | llm.with_structured_output(Questions)


In [300]:
question_chain = questions_prompt | llm.with_structured_output(Questions)

In [301]:
len(parent_documents)

11

In [302]:
embedding_dimension

768

In [303]:
for i, parent in enumerate(parent_documents):
    resp = question_chain.invoke(parent.page_content)
    if (resp != None):
        questions = resp.questions
        print(questions)
        print('-------')

['What are the potential impacts of climate change on businesses?']
-------
["What are Emirates Islamic's environmental goals?"]
-------
['What is the most popular sport among children?']
-------
['What are the 17 Sustainable Development Goals?']
-------
['What is the most popular sport among children?']
-------
['What is the percentage of operations assessed for risk related to corruption in 2023?']
-------


In [304]:
for i, parent in enumerate(parent_documents):
    resp = question_chain.invoke([parent.page_content])
    if (resp != None):
        questions = resp.questions
        params = {
            "parent_id": i,
            "questions": [
                {"text": q, "id": f"{i}-{iq}", "embedding": EMBEDDING_MODEL.embed_query(q)} #EMBEDDING_MODEL.get_embeddings([q])[0].values}
                for iq, q in enumerate(questions)
                if q
            ],
        }
        graph.query(
            """
        MERGE (p:Parent {id: $parent_id})
        WITH p
        UNWIND $questions AS question
        CREATE (q:Question {id: question.id})
        SET q.text = question.text
        MERGE (q)<-[:HAS_QUESTION]-(p)
        WITH q, question
        CALL db.create.setVectorProperty(q, 'embedding', question.embedding)
        YIELD node
        RETURN count(*)
        """,
            params,
        )
        # Create vector index #CALL db.index.vector.createNodeIndex
        try:
            graph.query(
                "CALL db.index.vector.createNodeIndex('hypothetical_questions', "
                "'Question', 'embedding', 768, 'cosine')",
                {},
            )
        except ClientError:  # already exists
            pass
#CALL db.index.vector.createNodeIndex('hypothetical_questions', 'Question', 'embedding', 768, 'cosine')

In [305]:


# Ingest summaries

summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            ("Generate a summary of the following input: {question}\n" "Summary:"),
        ),
    ]
)

summary_chain = summary_prompt | llm


In [306]:

for i, parent in enumerate(parent_documents):
    summary = summary_chain.invoke({"question": parent.page_content}).content
    params = {
        "parent_id": i,
        "summary": summary,
        "embedding": EMBEDDING_MODEL.embed_query(summary)# EMBEDDING_MODEL.get_embeddings([summary])[0].values,
    }
    graph.query(
        """
    MERGE (p:Parent {id: $parent_id})
    MERGE (p)-[:HAS_SUMMARY]->(s:Summary)
    SET s.text = $summary
    WITH s
    CALL db.create.setVectorProperty(s, 'embedding', $embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )
    # Create vector index
    #CALL db.index.vector.createNodeIndex('typical_rag', 'Parent', 'embedding', 768, 'cosine')
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('summary', "
            "'Summary', 'embedding', 768, 'cosine')",
            {},
        )
    except ClientError:  # already exists
        pass
# CALL db.index.vector.createNodeIndex('summary', 'Summary', 'embedding', 768, 'cosine')

In [307]:
# Typical RAG retriever
EMBEDDING_MODEL_NAME = 'textembedding-gecko@003'



typical_rag = Neo4jVector.from_existing_index(
    url = url,
    username = username,
    password = password,
    embedding = VertexAIEmbeddings(model_name=EMBEDDING_MODEL_NAME),
    index_name = 'typical_rag'
)


In [308]:

parent_query = """
MATCH (node)<-[:HAS_CHILD]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata LIMIT 1
"""

parent_vectorstore = Neo4jVector.from_existing_index(
    url = url,
    username = username,
    password = password,
    embedding = VertexAIEmbeddings(model_name=EMBEDDING_MODEL_NAME),
    index_name="parent_document",
    retrieval_query=parent_query,
)

In [309]:


# # Hypothetic questions retriever

hypothetic_question_query = """
MATCH (node)<-[:HAS_QUESTION]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata
"""

hypothetic_question_vectorstore = Neo4jVector.from_existing_index(
    url = url,
    username = username,
    password = password,
    embedding = VertexAIEmbeddings(model_name=EMBEDDING_MODEL_NAME),
    index_name="hypothetical_questions",
    retrieval_query=hypothetic_question_query,
)

In [310]:

# # # Summary retriever

summary_query = """
MATCH (node)<-[:HAS_SUMMARY]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata
"""

summary_vectorstore = Neo4jVector.from_existing_index(
    url = url,
    username = username,
    password = password,
    embedding = VertexAIEmbeddings(model_name=EMBEDDING_MODEL_NAME),
    index_name="summary",
    retrieval_query=summary_query,
)

In [311]:
retriever = typical_rag.as_retriever().configurable_alternatives(
    ConfigurableField(id="strategy"),
    default_key="typical_rag",
    parent_strategy=parent_vectorstore.as_retriever(),
    hypothetical_questions=hypothetic_question_vectorstore.as_retriever(),
    summary_strategy=summary_vectorstore.as_retriever(),
)

In [312]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [313]:
template = """Your are a financial analyst Answer the question based only on the following context:
{context}
Use this above information to answer queries about Environment, Social, and Governance Report
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [314]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(
    model="gemini-1.0-pro-vision",
    temperature=0,
    max_tokens=None,
    max_retries=6,
    stop=None,
    # other params...
)

In [315]:


chain = (
    RunnableParallel(
        {
            "context": itemgetter("question") | retriever | format_docs,
            "question": itemgetter("question"),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [316]:
# Add typing for input
class Question(BaseModel):
    question: str


chain = chain.with_types(input_type=Question)

In [317]:
original_query = "What is ESG Framework?"
print(
    chain.invoke(
        {"question": original_query},
        {"configurable": {"strategy": "summary_strategy"}},
    )
)

The ESG Framework is driven by the UN Sustainable Development Goals, the UAE Emirates NBD Vision, and the UN Global Compact Principles. It is reported using the GRI Standards, the Dubai Financial Market ESG Reporting Guide, and the IIRC Framework.


In [318]:
original_query = "Explain about Key Focus Area?"
print(
    chain.invoke(
        {"question": original_query},
        {"configurable": {"strategy": "parent_strategy"}},
    )
)

The key focus areas of the framework are Environment, Social, and Governance.


In [319]:
original_query = "Summarize Financial Literacy and Inclusion?"
print(
    chain.invoke(
        {"question": original_query},
        {"configurable": {"strategy": "hypothetical_questions"}},
    )
)

Emirates Islamic promotes financial literacy and inclusion through initiatives such as:

* Financial literacy programs for customers and the community
* Partnerships with educational institutions and NGOs
* Simplified financial products and services
* Digital banking platforms for easy access to financial services


In [320]:
original_query = "Summarize Financial Literacy and Inclusion?"
print(
    chain.invoke(
        {"question": original_query},
        {"configurable": {"strategy": "typical_rag"}},
    )
)

Emirates Islamic Bank is committed to expanding financial services for underserved groups. Initiatives include e-learning programs, customer education materials, and disability-friendly branches. The bank also partners with MBRHE to promote financial literacy in schools, universities, and government departments. Additionally, Emirates Islamic supports women entrepreneurs through the Businesswomen Account UAE.


In [321]:
original_query = "Sumaarize Feedback and Coaching:?"
print(
    chain.invoke(
        {"question": original_query},
        {"configurable": {"strategy": "parent_strategy"}},
    )
)

Feedback and Coaching: Emirates Islamic fosters a coaching culture to enhance interactions and organizational transformation. This involves providing feedback, identifying learning needs, and developing capabilities to support business strategies.
